In [0]:
%pip install openpyxl
import pandas as pd
import openpyxl
from pyspark.sql import SparkSession
import os
import numpy as np

In [0]:

# Path al archivo de interés 2025-2026.xlsx
xlsx_path = os.path.abspath(os.path.join("..", "source_files", "2027-2028.xlsx"))

# Cargar como dataframe
pdf = pd.read_excel(
    xlsx_path,
    header=None,
    dtype=str,
    na_filter=False
)


# Obtener el # de fila con las palabras ['2025', 'Cantidad Solicitada', 'Mínimo']
place_holders = ['IMSS', 'MIN']
# Obtener headers para la tabla Bronze
# 1. Obtenemos el índice basado para cada place holder
_p1 = pdf.index[pdf.iloc[:, 2] == place_holders[0]]
_p2 = pdf.index[pdf.iloc[:, 2] == place_holders[1]]

# 2. Extramos filas, reemplazando "" por nan
header_p1 = pdf.iloc[_p1].replace("", pd.NA)
header_p2 = pdf.iloc[_p2].replace("", pd.NA)


# 3. Llenamos celdas en blanco a partir de la columna 2
header_p1.iloc[:, 2:] = header_p1.iloc[:, 2:].ffill(axis=1)

# 4. Combinamos los dataframes y transponemos, para después combinarlos
df_headers = pd.concat([header_p1.T, header_p2.T,], axis=1)

headers_raw = df_headers.apply(
    lambda row: ' '.join(row.dropna().astype(str)) if row.notna().any() else np.nan,
    axis=1
)

# 5. Limpiamos 

vocales = str.maketrans({'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u'})

headers = (
    headers_raw  
    .astype(str)                            # ensure strings
    .str.strip()                            # trim whitespace
    .str.lower()                            # lowercase
    .str.replace("cantidad solicitada", "", regex=True)
    .str.replace(r"[\s+.]+", "_", regex=True)
    .str.translate(vocales)
)

#display(headers_clean)
# 6. Convertimos a lista
headers_clean = headers.tolist()

print(f"✅ Headers extraídos exitosamente: {headers_clean[0:3]}...")

In [0]:
# Ignoramos las primeras filas con información no relevante 
info_irrelevente = {'': np.nan, 'CLAVE': np.nan}
pdf.iloc[:, 0] = pdf.iloc[:, 0].replace(info_irrelevente)
# Retiramos los nan basados en la primera columna

data_pdf = pdf.dropna(subset=[pdf.columns[0]])

# Creamos el dataframe de spark
spark_df = spark.createDataFrame(data_pdf) \
                 .toDF(*headers_clean)

# Guardamos la tabla Delta
table = "workspace.default.bronze_2027_2028"
spark_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(table)

print(f"✅ Tabla e ingestión completada: {table}")